RF-DETR is a real-time, transformer-based object detection model architecture developed by Roboflow and released under the Apache 2.0 license.

More info: https://rfdetr.roboflow.com/

# Config env

In [10]:
# !pip install rfdetr supervision

### Check GPU availability

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


### Load model and check inference

Simply check if everything is working fine before more work

Runs inference on an example image using a pretrained RF-DETR Medium model (trained on COCO). To use a different model size, simply replace RFDETRMedium with RFDETRNano, RFDETRSmall, RFDETRBase or RFDETRLarge as needed.

In [8]:
# !wget -q https://media.roboflow.com/notebooks/examples/dog-2.jpeg
# !wget -q https://media.roboflow.com/notebooks/examples/dog-3.jpeg

In [9]:
# import numpy as np
# import supervision as sv

# from PIL import Image

# from rfdetr import RFDETRMedium
# from rfdetr.util.coco_classes import COCO_CLASSES

# image = Image.open("dog-2.jpeg")

# model = RFDETRMedium(resolution=640)
# # model.optimize_for_inference()

# detections = model.predict(image, threshold=0.5)

# color = sv.ColorPalette.from_hex([
#     "#ffff00", "#ff9b00", "#ff8080", "#ff66b2", "#ff66ff", "#b266ff",
#     "#9999ff", "#3399ff", "#66ffff", "#33ff99", "#66ff66", "#99ff00"
# ])
# text_scale = sv.calculate_optimal_text_scale(resolution_wh=image.size)
# thickness = sv.calculate_optimal_line_thickness(resolution_wh=image.size)

# bbox_annotator = sv.BoxAnnotator(color=color, thickness=thickness)
# label_annotator = sv.LabelAnnotator(
#     color=color,
#     text_color=sv.Color.BLACK,
#     text_scale=text_scale,
#     smart_position=True
# )

# labels = [
#     f"{COCO_CLASSES[class_id]} {confidence:.2f}"
#     for class_id, confidence
#     in zip(detections.class_id, detections.confidence)
# ]

# annotated_image = image.copy()
# annotated_image = bbox_annotator.annotate(annotated_image, detections)
# annotated_image = label_annotator.annotate(annotated_image, detections, labels)
# annotated_image.thumbnail((800, 800))
# annotated_image

# Prepare data